# AWS를 활용한 인공지능 모델 배포

## [contents]
### 1. 클라우드 기초
### 2. 실습 : AWS & 실습 환경 세팅
### 3. API to serve ML model
### 4. 실습 : Serialization & De-serialization
### 5. 실습 : Inference를 위한 model handler개발
### 6. Flask 기반 감성분석 API 개발



----
## 1. `클라우드` 기초
- 클라우드 = 인터넷 기반 컴퓨팅의 일종
### 1-1) 클라우드를 `사용하는 이유와 그 예시`
- 인터넷 사용자가 크게 증가하고 다양한 서비스를 제공하게 되면서 필요한 때에 필요한 만큼 서버를 증설하기 원하는 `온디맨드 수요 증가` (클라우드를 통해서 적은 비용으로 빠르게 필요한 IT 환경(빅데이터, 플랫폼, 인고지능 개발 등의 환경 마련 가능)
- ex) `인공지능 서비스 제공` : `도커와 같은 가상화 기술`을 통해 `GPU 활용과 소프트웨어 설치 및 배포 등의 작업에 비용과 시간 절감 가능
- ex) `속도-주문형 셀프 서비스` : 클라우드 제공자와 `별도의 커뮤니케이션 없이` 원하는 클라우드 서비스를 바로 이용 가능
- ex) `접근성` : 사용자의 위치, 시간과 관계없이 어떤 디바이스로도 접근 가능
- ex) `확장성` : 이용량 증가시 유연하게 추가 확장 가능
- ex) `생산성`, `측정 가능성` : 하드웨어, 소프트웨어 설치 비용 절감, 클라우드 서비스를 사용한 만큼만 과금
- ex) `보안`, `안정성` : 클라우드 공급자가 전체적으로 보안이나 안정성 제공

### 1-2) 클라우드 컴퓨팅 `운용 모델`
#### `구축 및 배포 유형`에 따라 세 가지 형태로 구분
#### `Public`, `Private`, `Hybrid` (분산형 등 다양한 형태가 존재함, 최근에는 네이버, 아마존, 마소 등 여러 클라우드를 공유하여 사용하는 분산형 클라우드도 블록체인에서 사용되고 있음)

#### [`Public`] 
##### - `서비스 유지`를 위한 모든 인프라와 IT기술을 클라우드에서 사용
##### - AWS, GCP, Azure와 같은 외부 클라우드 컴퓨팅 사업자가 IT자원을 소유하고 인터넷을 통해 제공
##### - IT관리 `인력이나 인프라 구축 비용이 없는 경우`에 유용

#### [`Private`] 
##### - 고객이 `자체 데이터 센터에서 직접 클라우드 서비스를 구축`하는 형태
##### - 보안이 좋고 customization이 가능하며 판매자는 IT기술만 패키지 형태로 판매하기도 함

#### [`Hybrid`] 
##### - 고객의 핵심 시스템은 내부에 두면서도 외부의 클라우드를 활용하는 형태
##### - 퍼블릭의 경제성과 프라이빗의 보안성을 모두 고려

### 1-3) 클라우드 서비스 `제공 모델`
#### `클라우드 서비스 제공 방식`에 따라 세 가지 형태로 구분
#### `IaaS`, `PaaS`, `SaaS` (`On-Premises` : 하드웨어, 클라우드, 소프트웨어어플리케이션 등을 모두 관리하는 모델)

#### [`IaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data`, `Runtime`, `Middleware`, `OS` 직접 관리
##### - `Virtualization`, `Servers`, `Storage`, `Networking` 같은 인프라 측면은 빌림

#### [`PaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data` 직접 관리
##### - `Runtime`, `Middleware`, `OS`, `Virtualization`, `Servers`, `Storage`, `Networking` 플랫폼, 인프라 측면을 빌림

#### [`SaaS`] : 인프라 측면만 빌리고 나머지는 직접 관리
##### - `App`, `Data`, `Runtime`, `Middleware`, `OS`, `Virtualization`, `Servers`, `Storage`, `Networking` 서비스, 플랫폼, 인프라 측면을 모두 빌림
##### - ex) office365

- 다양한 사업자들 : AWS, GCP, Azure, NCP
- 이번 실습에서는 `Amazon EC2`를 사용할 예정

----
## 2. 실습 : AWS & 실습 환경 세팅

### 2-1) AWS 계정 가입 (https://aws.amazon.com/ko/)
- 실습 환경 세팅은 영상 및 학습 보고서 참고하기 

----
## 3. API to serve ML model
- 본 실습에서는 `AWS EC2`와 `Python Flask`를 기반으로 모델 학습 및 추론을 요청/응답하는 `API 서버 개발`을 만드는 것이 목표

![api_sever](./pic/api_sever.png)
- API Server with DL

### 3-1) `Interface`
- 상호 `합의된 매뉴얼`에 따라 적절한 `입력에 따른 출력이 제공`하는 제품
- ex) 마우스, 키보드 : 입력을 위한 인터페이스
- ex) 모니터, 프린터 : 정보를 받는 출력을 위한 인터페이스

### 3-2) `API` (Application Programming Interface)
- `노드`와 `노드`(기계와 기계, sw와 sw)간 데이터를 주고 받는 `커뮤니케이션을 위한 인터페이스` (합의된 입력에 따른 출력)

### 3-3) `RESTful API`
- REST 아키텍쳐를 따르는 API. (`HTTP URI`, `HTTP Method`를 통해 필요한 연산을 요청, 반황하는 API)
- 일반적으로 데이터 값을 담아 요청하고 모델이 추론한 결과에 대한 return을 `json 형태로 반환`하도록 설계
- RESTful API는 `요청 메시지만 봐도 어떤 내용인지` 알 수 있도록 표현됨

### 3-4) `Practical process of ML` 
- `문제 정의` -> `Data 준비`, `Build and Train Model` ->  `Deploy Model`, `Monitor/Scale in app`
- 실습에서는 저장된 `Model File을 불러와서 Run/Predict` 하는 것이 목표

### 3-5) `Model Serving`
- `Model Training` -> `Serializing Model`(학습된 모델을 저장) -> `Serving Model`
- 이때 주의할 점은 `모든 과정이 연속성이 유지`되어야한다. (데이터 처리 방식과 데이터 분포가 서로 다르다면 기능이 떨어질 수 있다.)
- 상황에 맞는 다양한 Framework를 통해 serving을 해야함.
- framework ex) `Tensor Flow serving`, `TorchServe`, `Tensor RT` 등

### 3-6) `Serialization`
- 학습한 모델을 저장하고 불러오는 것
- `Serialization` : Memory에서 Disk로 저장
- `De-serialization` : Disk에 있는 정보를 Memory로 불러 읽어옴
- 두 방식은 동일해야함

### 3-7) `Skeleton of handler to serve model`
- 사람이 `모델의 결과 값을 잘 읽기 위해` 데이터를 `레이블링` (후처리) 하는 과정

----
## 4. kdt ai aws 실습 (감정을 분석하는 Model) 
#### 4-2) Model Training -> 4-3) Serializing Model -> 4-4) Serving Model

### 4-1) 실습 환경 세팅

### 4-2) Model Training
- Data preprocessing
- Model fitting
- Evaluation

In [ ]:
# aws EC2에서 아나콘다 가상환경 실행
%conda activate pytorch_p39

# 가상환경이 제대로 실행됐는지 확인
%python
%import torch
%torch.__version__
%exit()

# template 소스코드 다운로드
%git clone https://github.com/sackoh/kdt-ai-aws

# 다운받은 파일의 경로로 이동
%ls
%cd ./kdt-ai-aws/

# 필요 라이브러리 설치
%pip install -r requirements.txt

# 설치 내용 확인
%pip list

### 4-3) Serializing Model
- Save trained model

In [ ]:
# clone한 모델 파일 경로로 파일을 열고 아나콘다 환경 실행
%conda activate pytorch_p39

# 학습 시행 및 모델 생성, 저장
%python train_ml.py

#### 모델 학습 후 `model 파일이 생성`되며 해당 파일에서 `Serialization 코드`를 확인할 수 있다.

![check_serialization](./pic/check_serialization.png)

### 4-4) Serving Model
- Load Trained model (De-serialization)
- 저장된 모델을 불러와 특정 입력값에 대한 예측 수행
- 터미널, .py, .ipynb 파이렝서 모두 아래 코드 실행 가능

In [ ]:
# pythorch 환경에서 python 실행
%python

# 모델 불러오고
# -> '재밌는 영화입니다.'input에 대해서 vectorizer가 어떻게 text를 벡터로 표현한 후
# -> 표현된 벡터를 model이 input으로 받았을 때 나오는 정보를
# -> 후처리하는 과정

# De-serialization (모델 불러오기)
%import joblib
%model = joblib.load('model/ml_model.pkl')

# model 확인
%model

# vectorizer(음절 카운트 등의 기능이 있음) 불러오기
%vectorizer = joblib.load('model/ml_vectorizer.pkl')

# vectorizer 확인
%vectorizer

In [ ]:
# vectorizer 사용
%text = '재미있는 영화입니다.'
%text
%model_input = vectorizer.transform([text])
%model_input

# model_input 안에 text 내용이 있는지 확인
%print(model_input.asformat('array'))

# 해당 input을 model 안에 넣기
%model_output = model.predict_proba(model_input)

# cf. model method 확인
%dir(model)

In [ ]:
# 후처리 작업 (전처리 방식과 동일해야함)
%model_output = model_output.argmax(axis=1)
%model_output

# model이 내린 감정 판단을 사람이 볼 수 있는 언어로 데이터 후처리
%id2label = {0:'netive', 1:'positive'}

%print(f'sentiment : {id2label[model_output[0]]}')

# 모델 불러오고
# -> '재밌는 영화입니다.'input에 대해서 vectorizer가 어떻게 text를 벡터로 표현한 후
# -> 표현된 벡터를 model이 input으로 받았을 때 나오는 정보를
# -> 후처리하는 과정

----
## 5. kdt ai aws 실습 : `Inference`(전처리)를 위한 `model hamdler` 개발 (4-4) 이어서)
- 입력된 데이터에 대해서 예측을 수행하는 model hamdler 개발

### 5-1) Serving Model
- Load Trained model (4번 내용)
- `Define inference` (이번 내용)

#### `handle()` : 요청 정보를 받아 적절한 응답을 반환
#### 1. 정의된 양식으로 데이터가 입력됐는지 확인
#### 2. 입력 값에 대한 전처리 및 모델에 입력하기 위한 형태로 변환
#### 3. 모델 추론
#### 4. 모델 반환값의 후처리 작업
#### 5. 결과 반환

In [ ]:
# model.py 파일에서 실습

def handle(self, data):
    # 입력된 data 전처리
    model_input = self.preprocess(data)
    # 모델을 불러와서 input에 대한 모델 추론
    model_output = self.preprocess(model_input) 
    # 모델의 output을 반환
    return self.postprocess(model_output)


#### `initialize()` : 데이터 처리나 모델, configuration 등 초기화
#### 1. Configuration 등 초기화
#### 2. (Optional) 신경망을 구성하고 초기화
#### 3. 사전 학습한 모델이나 전처리기 불러오기 (De-serialization)

- 모델은 `전역변수`로 불러와야 합니다. 만약 inference를 할 때마다 모델을 불러오도록 한다면 그로 인해 발생하는 시간이나 자원 등의 낭비가 발생함

In [ ]:
# model.py 파일에서 실습
# class MLModelHandler(ModelHandler):

def initialize(self, ):
    import joblib
    self.model = joblib.load('model/ml_model.pkl')
    self.vectorizer = joblib.load('model/ml_vectorizer.pkl')

#### `preprocess()` : Raw input을 전처리 및 모델 입력 가능형태로 변환
#### 1. Raw input 전처리 (데이터 클린징의 목적과 학습된 모델의 학습 당시 scaling이나 처리 방식과 맞춰주는 것이 필요)
#### 2. 모델에 입력가능한 형태로 변환 (vectorization, convertion to id 등의 작업)

In [ ]:
# model.py 파일에서 실습
# class MLModelHandler(ModelHandler):

def preprocess(self, text):
    # cleansing raw text
    model_input = self._clean_text(text)

    # vectorizing cleaned text
    model_input = self.vectorizer.transform(model_input)
    return model_input

#### `inference()` : 입력된 값에 대한 예측/ 추론
#### 1. 각 모델의 predict 방식으로 예측 확률분포 값 반환

In [ ]:
# model.py 파일에서 실습
# class MLModelHandler(ModelHandler):

def inference(self, model_input):
    # get predictions from model as probabilities
    model_output = self.model.predict_proba(model_input)
    return model_output

#### `postprocess()` : 모델의 예측값을 response에 맞게 후처리 작업
#### 1. 예측된 결과에 대한 후처리 작업
#### 2. 보통 모델이 반환하는 건 확률분포와 같은 값이 기 떄문에 response에서 받아야하는 정보로 처리하는 역할을 많이 함

In [ ]:
# model.py 파일에서 실습
# class MLModelHandler(ModelHandler):

def postprocess(self, model_output):
    predicted_probabilities = model_output.max(axis=1)
    predicted_ids = model_output.argmax(axis=1)
    predicted_labels = [self.id2label[id_] for id_ in predicted_ids]
    return predicted_labels, predicted_probabilities

- 위 세팅을 테스트하기 위한 model import

In [ ]:
%python

%from model import MLModelHandler
%ml_handler = MLModelHandler()

# model, vectorizer가 잘 불러와졌는지 확인
%dir(ml_handler)
%ml_handler.model
%ml_handler.vectorizer

In [ ]:
# modle test
%text = ['정말 재미있는 영화입니다.', '정말 재미가 없습니다.']

%result = ml_handler.handle(text)

----
## 6. kdt ai aws 실습 : `Flask`기반 감성분석 `API` 개발
- 

### 6-1) 네이버 영화리뷰 감성분석 개요
- 네이버 영호리뷰 데이터로 학습한 ML/DL 모델을 활용해 `감성분석 API` 개발
- `나이브베이지안 모델`과 `딥러닝 모델`로 학습한 두 개의 모델을 서빙하며 0은 부정을 1은 긍정을 의미한다.
- 학습에 사용한 `데이터는` 박은정님이 공개한 `NSMC 데이터`

### 6-2) 감성분석 API 개발 방향
- AWS EC2와 python Flask 기반 모델 학습 및 추론을 요청/응답하는 API 서버 개발

![mvreview_ML_API](./pic/mvreview_ML_API.png)

### 6-3) API 정의 (요청 방식과 반환 방식 정의)
- 요청 방식 : `POST` 방식으로 `predict 요청`, `value 형태`의 `json 포맷`으로 요청(do_fast를 true로 할 경우, 빠른 추론 속도가 가능한 머신 러닝 모델로 추론하도록 함. false의 경우, 추론 속도는 비교적 느리지만 정확도가 높은 딥러닝 모델로 추론하도록 함)
- 반환 방식 : `text index` 별로 `key : value`로 결과를 저장한 `json 포맷`으로 결과 반환

### 6-3) Add DL model handler
- 사전 학습한 딥러닝 모델 (Hugging Face에서 제공하는 외부 저장소에서 다운) 을 활용하여 `머신러닝 모델 handler`와 `동일한 입력`에 대해 `동일한 결과`를 반환하는 `handler 개발`

In [ ]:
# model.py DL Model Handler 개발
import torch

# class DLModelHandler(ModelHandler):
def initialize(self, ):
    from transformers import AutoTokenizer, AutoModleForSequenceClassification
    self.model_name_or_path = 'sackoh/bert-base-multiligual-cased-nsmc'
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path)
    self.modle = AutoModleForSequenceClassification.from_pretrained(self.model_name_or_path)
    self.model.to('cpu')

def preprocess(self, text):
    # preprocess raw text
    model_input = self._clean_text(text)
    
    # vectorizer cleaned text
    model_input = self.tokenizer(text, return_tensors='pt', padding=True)
    return model_input

def inference(self, model_input):
    # 그레디언트 값을 저장하지 않겠다는 함수
    with torch.no_grad():
        # modle로부터 입력값을 집어넣고 결과를 반환하는 과정
        model_output = self.model(**model_input)[0].cpu()
        model_output = 1.0 / (1.0 + torch.exp(-model_output))
        model_output = model_output.numpy().astype(float)

    return model_output

def postprocess(self, model_output):
    predicted_probabilities = model_output.max(axis=1)
    predicted_ids = model_output.argmax(axis=1)
    predicted_labels = [self.id2label[id_] for id_ in predicted_ids]

    return predicted_labels, predicted_probabilities

def handler(self, data):
    model_input = self.preprocess(data)
    model_output = self.inference(model.input)

    return self.postprocess(model_output)

### 6-4) Unittest model handlers
- 위 개발한 model handlers가 잘 동작하는지 unittest를 해주는 것은 좋은 습관이다.

In [ ]:
# test_model_handler.py 파일 생성
import unittest
from model import MLModelHandler, DLModelHandler

text = ['음악이 주가 된, 최고의 음악영화',
'발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네',
'실화라서더욱아름답고찡하네요...많이울었어요벌써4년이라는시간이흘렀네요']

class TestModelHandler(unittest.TestCase):
    def test_ml_model_handler(self):
        predicted = ['positive', 'negative', 'negative']
        ml_handler = MLModelHandler()
        results = ml_handler.handle(text)

        for label, pred in zip(results[0], predicted):
            self.assertEqual(label, pred)

    def test_dl_model_handler(self):
        predicted = ['positive', 'negative', 'positive']
        dl_handler = DLModelHandler()
        results = dl_handler.handle(text)

        for label, pred in zip(results[0], predicted):
            self.assertEqual(label, pred)

if __name__ == '__main__':
    unittest.main()

In [ ]:
# 아나콘다 가상환경을 켠 후 unittest를 진행
%python -m unittest -v test_model_handler.py

### 6-5) Flask API 개발 & 배포
- `Model을 전역변수로 불러오고` 요청된 텍스트에 대해 `예측 결과를 반환하는 코드` 입력

In [ ]:
# app.py 파일에서

# assign model handler as global variable [2 LINES]
ml_handler = MLModelHandler()
dl_handler = DLModelHandler()

# def predict():
if do_fast:
    predictions = ml_handler.handler(text)
else:
    predictions = dl_handler.handler(text)

# {'text': t, 'label': l, 'confidence': c}인 key value 값를 저장하고
# key를 index값인 str(i)로 하여 
# 최종적으로 {} json 형태를 반환
result = json.dumps({str(i): {'text': t, 'label': l, 'confidence': c}
                        for i, (t, l, c) in enumerate(zip(text, predictions[0], predictions[1]))})
return result

### 6-6) Test API on remote
- 원격에서 서버로 API에 요청하여 테스트 수행

In [ ]:
%ll

# app.py 파일 실행
%python app.py

# 5000포트로 서비스가 도는 것을 확인할 수 있음

- host : EC2 인스턴스 생성 시에 받은 퍼블릭 IP 주소 또는 직접 할당한 탄력적 IP 주소 (ex. 54.83.1.129)
- post : EC2 인스턴스 생성 시에 설정했던 port 번호 (ex. 5000)

In [ ]:
# 호출 첫번째 방식 (새로운terminal 환경)

# -> kdt
%curl -d '{"text": ["영화 오랜만에 뵜는데 괜찮은 영화였어", "정말 지루했어"], "use_fast": false}' \
%-H "Content-Type: application/json" \
%-X POST \
%http://15.164.180:5000/predict

In [ ]:
# 호출 두번째 방식 (새로운 terminal 환경에서 python3로)

# -> kdt
%python3
%import requests
%url = "http://15.164.180:5000/predict"
%data = {"text": ["영화 오랜만에 뵜는데 괜찮은 영화였어", "정말 지루했어"], "use_fast": false}
%response = request.post(url, json=data)
%print(respons.content)